##  宽域对数粗扫 (Broadband Logarithmic Coarse Scan)

In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigs
from scipy.stats import linregress
import mpmath
import time
from numba import njit

# ================== 1. 真实的黎曼零点 (前 6 个绝密坐标) ==================
mpmath.mp.dps = 15
N_ZEROS = 100
TRUE_ZEROS = np.array([float(mpmath.zetazero(i).imag) for i in range(1, N_ZEROS + 1)])

TARGETS = TRUE_ZEROS[:6]

# ================== 2. 高斯核概率溅射引擎 ==================
@njit
def run_universe_sniper(steps, n_bins, u_c, k_opt, c_offset, eps):
    transitions = np.zeros((n_bins, n_bins), dtype=np.float64)
    V = np.zeros(n_bins, dtype=np.float64)
    
    dx = 2.0 / n_bins
    init_bin = int((0.5 + 1.0) / dx)
    if init_bin >= n_bins: init_bin = n_bins - 1
    elif init_bin < 0: init_bin = 0
    V[init_bin] = 1.0 
    
    inv_2eps2 = 1.0 / (2.0 * eps**2)
    radius = int(5.0 * eps / dx) + 1 
    
    for n in range(1, steps + 1):
        mu_raw = u_c + k_opt / (np.log(n + c_offset)**2.0)
        
        if mu_raw > 2.0: mu = 2.0
        elif mu_raw < 0.1: mu = 0.1
        else: mu = mu_raw
            
        V_next = np.zeros(n_bins, dtype=np.float64)
        
        for i in range(n_bins):
            if V[i] < 1e-12: continue
                
            x = -1.0 + dx * 0.5 + i * dx  
            x_next = 1.0 - mu * x * x     
            
            j_center = int((x_next + 1.0) / dx)
            j_start = max(0, j_center - radius)
            j_end = min(n_bins - 1, j_center + radius)
            
            w_sum = 0.0
            for j in range(j_start, j_end + 1):
                cj = -1.0 + dx * 0.5 + j * dx
                dist_sq = (cj - x_next) * (cj - x_next)
                w_sum += np.exp(-dist_sq * inv_2eps2)
            
            if w_sum > 1e-18:
                inv_w_sum = 1.0 / w_sum
                for j in range(j_start, j_end + 1):
                    cj = -1.0 + dx * 0.5 + j * dx
                    dist_sq = (cj - x_next) * (cj - x_next)
                    prob = np.exp(-dist_sq * inv_2eps2) * inv_w_sum
                    flow = V[i] * prob
                    V_next[j] += flow
                    transitions[i, j] += flow
            else:
                if j_center < 0: j_center = 0
                if j_center >= n_bins: j_center = n_bins - 1
                flow = V[i]
                V_next[j_center] += flow
                transitions[i, j_center] += flow
                
        V = V_next 
        
    return transitions

# ================== 3. 狙击任务配置 ==================
MU_END = 1.5437
TOTAL_STEPS = 10**6    
C_OFFSET = 10.0   
DELTA_MU_ABS = 0.02    

eps_array = np.geomspace(0.0001, 0.01, 20)
best_eps = 0
min_error_sum = 999.0

print(f"🎯 启动【前 6 零点防抖动雷达】| 正向频率 + 比例缩放")
print(f"真实坐标 => Z1:{TARGETS[0]:.2f} | Z2:{TARGETS[1]:.2f} | Z3:{TARGETS[2]:.2f} | Z4:{TARGETS[3]:.2f} | Z5:{TARGETS[4]:.2f} | Z6:{TARGETS[5]:.2f}")
print("=" * 90)

start_total_t = time.time()

for eps in eps_array:
    t_start_val = 1.0 / (np.log(1 + C_OFFSET)**2)
    t_end_val   = 1.0 / (np.log(TOTAL_STEPS + C_OFFSET)**2)
    k_opt = DELTA_MU_ABS / (t_start_val - t_end_val)
    u_c = MU_END - k_opt * t_end_val
    
    trans = run_universe_sniper(TOTAL_STEPS, 5000, u_c, k_opt, C_OFFSET, eps)
    
    P_sparse = sp.csr_matrix(trans, dtype=np.float64)
    row_sums = np.array(P_sparse.sum(axis=1)).flatten()
    row_sums[row_sums == 0] = 1.0 
    P_sparse.data /= row_sums[P_sparse.indices]
    
    try:
        # 为了确保能抓到足够多的正向特征值，k 设大一点
        eigenvalues, _ = eigs(P_sparse, k=N_ZEROS*2 + 20, which='LM', tol=1e-4)
        
        # 🎯 核心修复：只取复平面上半区（正向频率）
        pos_eigs = eigenvalues[eigenvalues.imag > 1e-4]
        phases = np.sort(np.angle(pos_eigs))
        
        min_len = min(len(phases), N_ZEROS)
        
        if min_len >= 6:
            phases_trunc = phases[:min_len]
            true_zeros_trunc = TRUE_ZEROS[:min_len]
            
            # 🎯 绝对纯净的比例缩放
            scale_factor = TARGETS[0] / phases_trunc[0]
            pred_zeros = phases_trunc * scale_factor
            
            errs = np.abs(pred_zeros[:6] - TARGETS)
            err_sum_2_to_6 = np.sum(errs[1:6])
            
            ssr = np.sum((true_zeros_trunc - pred_zeros)**2)
            sst = np.sum((true_zeros_trunc - np.mean(true_zeros_trunc))**2)
            r2 = max(0.0, 1.0 - (ssr / sst))
            
            if err_sum_2_to_6 < min_error_sum:
                min_error_sum = err_sum_2_to_6
                best_eps = eps
                
            print(f"ε = {eps:<8.5f} | R²: {r2:.4f} | ErrSum(Z2-Z6): {err_sum_2_to_6:.4f}")
            print(f"  ▶ 预测 Z: {pred_zeros[0]:.2f} | {pred_zeros[1]:.2f}({errs[1]:.2f}) | {pred_zeros[2]:.2f}({errs[2]:.2f}) | {pred_zeros[3]:.2f}({errs[3]:.2f}) | {pred_zeros[4]:.2f}({errs[4]:.2f}) | {pred_zeros[5]:.2f}({errs[5]:.2f})")
            print("-" * 90)
            
        else:
            print(f"ε = {eps:<8.5f} | 崩盘 (正向特征值不足)")
            print("-" * 90)
            
    except Exception:
        print(f"ε = {eps:<8.5f} | eigs 求解失败")
        print("-" * 90)

print("=" * 90)
print(f"👑 狙击完成！耗时 {(time.time()-start_total_t)/60:.2f} 分钟。")
print(f"🎯 最佳神圣底噪: ε = {best_eps:.5f} (Z2~Z6 总误差最小: {min_error_sum:.4f})")

🎯 启动【前 6 零点防抖动雷达】| 正向频率 + 比例缩放
真实坐标 => Z1:14.13 | Z2:21.02 | Z3:25.01 | Z4:30.42 | Z5:32.94 | Z6:37.59
ε = 0.00010  | R²: 0.0000 | ErrSum(Z2-Z6): 129.3713
  ▶ 预测 Z: 14.13 | 27.19(6.16) | 42.57(17.55) | 47.12(16.69) | 74.64(41.70) | 84.85(47.26)
------------------------------------------------------------------------------------------
ε = 0.00013  | R²: 0.0000 | ErrSum(Z2-Z6): 337.5683
  ▶ 预测 Z: 14.13 | 51.10(30.08) | 65.96(40.95) | 83.48(53.05) | 134.42(101.48) | 149.59(112.01)
------------------------------------------------------------------------------------------
ε = 0.00016  | R²: 0.0000 | ErrSum(Z2-Z6): 511.1966
  ▶ 预测 Z: 14.13 | 54.53(33.51) | 98.63(73.62) | 130.31(99.88) | 169.13(136.20) | 205.57(167.99)
------------------------------------------------------------------------------------------
ε = 0.00021  | R²: 0.0000 | ErrSum(Z2-Z6): 1129.8741
  ▶ 预测 Z: 14.13 | 92.50(71.48) | 190.14(165.13) | 262.80(232.38) | 313.71(280.78) | 417.70(380.11)
-----------------------------------

In [ ]:
==========================================================================================
ε = 0.00379  | R²: 0.0000 | ErrSum(Z2-Z6): 22.6951
  ▶ 预测 Z: 14.13 | 16.99(4.03) | 27.91(2.90) | 33.61(3.19) | 38.55(5.62) | 44.54(6.96)
------------------------------------------------------------------------------------------
ε = 0.00483  | R²: 0.0000 | ErrSum(Z2-Z6): 163.2036
  ▶ 预测 Z: 14.13 | 25.21(4.19) | 64.86(39.85) | 70.49(40.06) | 74.73(41.80) | 74.89(37.30)
------------------------------------------------------------------------------------------
ε = 0.00616  | R²: 0.0000 | ErrSum(Z2-Z6): 148.1928
  ▶ 预测 Z: 14.13 | 33.55(12.53) | 47.19(22.18) | 62.24(31.82) | 73.10(40.16) | 79.09(41.50)
------------------------------------------------------------------------------------------
ε = 0.00785  | R²: 0.3990 | ErrSum(Z2-Z6): 14.6247
  ▶ 预测 Z: 14.13 | 17.63(3.39) | 22.79(2.22) | 30.61(0.19) | 30.70(2.23) | 30.99(6.60)
------------------------------------------------------------------------------------------
ε = 0.01000  | R²: 0.0000 | ErrSum(Z2-Z6): 17.6038
  ▶ 预测 Z: 14.13 | 17.84(3.18) | 29.62(4.61) | 33.79(3.36) | 35.04(2.10) | 41.93(4.35)
------------------------------------------------------------------------------------------
==========================================================================================
👑 狙击完成！耗时 995.72 分钟。
🎯 最佳神圣底噪: ε = 0.00785 (Z2~Z6 总误差最小: 14.6247)